# Segmenting and Clustering Neighborhoods in Toronto

## 1. Scrape Data from Wikipedia page and creat a dataframe

#### Scrape Data from Wikipedia page

In [4]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import numpy as np
import pandas as pd

In [5]:
html = urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").read()

In [6]:
soup = BeautifulSoup(html, features='lxml') ###read with lxml format
type(soup)

bs4.BeautifulSoup

In [7]:
table = soup.find('table','wikitable sortable')
table0 = table.find_all('tr')[0].find_all('th')[0].get_text() # use lxml read header of table
table0

'Postcode'

In [8]:
table1 = table.find_all('tr')[1].find_all('td')[2].get_text()# use lxml read body of table
table1

'Not assigned\n'

#### Read table infor into a list table_NB

In [9]:
table_NB = []
for i,text in enumerate(table.find_all('tr')):
    if i == 0:
        for j in [0,1,2]:
            table_NB.append(table.find_all('tr')[i].find_all('th')[j].get_text().replace('\n',''))
    else:
        brgh = table.find_all('tr')[i].find_all('td')[1].get_text().replace('\n','')
        if brgh != 'Not assigned': # not reading 'Not assigned' infor
            for j in [0,1,2]:
                table_NB.append(table.find_all('tr')[i].find_all('td')[j].get_text().replace('\n',''))

        

#### Format table_NB into pd.DataFrame

In [10]:
table_NB = np.array(table_NB)
table_NB = table_NB.reshape(-1,3)
table_NBpd = pd.DataFrame(table_NB)

In [11]:
columns = table_NBpd.loc[0,]
table_NBpd.columns = columns

In [12]:
table_NBpd = table_NBpd.drop([0])

In [13]:
table_NBpd = table_NBpd.reset_index(drop=True)
table_NBpd.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [14]:
table_NBpd.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object', name=0)

#### Find 'Not assigned' Neighbourhood and use Borough's value

In [15]:
a = table_NBpd.loc[table_NBpd['Neighbourhood']=='Not assigned']

In [16]:
table_NBpd.at[a.index[0],'Neighbourhood']=table_NBpd.at[a.index[0],'Borough']

In [22]:
table_final = table_NBpd.sort_values('Postcode').groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: "%s" % ','.join(x)).reset_index()
table_final.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [24]:
table_final.shape

(103, 3)

In [91]:
table_final.to_csv('part1.csv')

## Conclusion: 
This assignment used Beautifulsoup to scrape Toronto's borough and neighbourhood information from wikipedia. 
With the help of panda DataFrame library, I replaced Neighbourhood 'Not assigned' value to Borough' value.  
In the end, I printed shape of final table.